In [1]:
#| default_exp agents.situated

In [2]:
#| export

import numpy as np
from projective_simulation.agents.abstract_PS import PS_Agent
from projective_simulation.ECMs.priming import Priming_ECM
from projective_simulation.ECMs.episodic import Episodic_Memory
from projective_simulation.methods.preprocessors import action_factorizor

class Situated_Agent(PS_Agent):
    def __init__(self, 
                 reflex_ECM = None, #if an ECM object is not given, a number of actions must be given with which to create one
                 episode_ECM = None,
                 num_actions = None, # The number of available actions. If an ECM is not given, should be int
                 memory_capacity:int = None,
                 glow: float = 0.1, # The glow (or eta) parameter. Won't be used if ECM is given
                 damp: float = 0., # The damping (or gamma) parameter. Won't be used if ECM is given
                 reflex_softmax: float = 0.1, # The softmax (or beta) parameter. Won't be used if ECM is given
                 PS_softmax: float = 0.7,
                 focus: float = 0,
                 error_tolerance: float = 0.01,
                 min_expectation: float = 0.01,
                 deliberation_length: int = 1,
                 t = 0,
                 percept_processor = None, 
                 action_processor = None
                ):
        assert isinstance(reflex_ECM, Priming_ECM) or isinstance(num_actions, int)
        if reflex_ECM is None:
            self.reflex_ECM = Priming_ECM(num_actions, glow, damp, reflex_softmax)
        else:
            self.reflex_ECM = reflex_ECM

        assert isinstance(episode_ECM, Episodic_Memory) or isinstance(memory_capacity, int)
        if episode_ECM is None:
            self.episode_ECM = Episodic_Memory(num_actions = self.reflex_ECM.num_actions, 
                                               capacity = memory_capacity, 
                                               softmax = PS_softmax,
                                               focus = focus,
                                               error_tolerance = error_tolerance,
                                               min_expectation = min_expectation,
                                               deliberation_length = deliberation_length,
                                               t = t
                                              )
        else:
            self.episode_ECM = episode_ECM
        assert self.reflex_ECM.num_actions == self.episode_ECM.num_actions
        self.num_actions = self.reflex_ECM.num_actions

        if percept_processor is None:
            self.percept_processor = action_factorizor(num_actions = self.num_actions)
        else:
            self.percept_processor = percept_processor

    def get_action(self, observation):
        action = self.reflex_ECM.deliberate(str(observation))
        percept = self.percept_processor.get_percept(observation, action)
        self.episode_ECM.deliberate(percept) #runs predictions, priming actions for next step.
        return(action)

    def update(self):
        reward = np.nanmean(self.episode_ECM.valences) - self.episode_ECM.surprise
        self.ECM.learn(reward)
                                               
                                               


In [7]:
test = Situated_Agent(num_actions = 2, memory_capacity = 10)


0

In [13]:
test.get_action(np.array([5,1]))
test.episode_ECM.trace_encoder

array([[ True, False,  True, False,  True,  True, False, False, False,
        False],
       [False,  True, False,  True, False, False, False, False, False,
        False],
       [ True,  True,  True,  True, False, False, False, False, False,
        False],
       [ True,  True,  True, False, False, False, False, False, False,
        False],
       [False, False, False,  True,  True,  True, False, False, False,
        False],
       [False, False, False, False,  True, False, False, False, False,
        False],
       [False, False, False, False, False,  True, False, False, False,
        False]])